In [1]:
import pandas as pd
import ast
import sys
from pathlib import Path
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from nltk.metrics import jaccard_distance
from nltk.util import ngrams
from nltk.wsd import lesk
from collections import Counter
import math
from itertools import chain
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Add the project directory to the Python path
project_dir = Path.cwd().parent
sys.path.append(str(project_dir))

from Preprocessing.preprocessingUtils import TextPreprocessor

# Ensure necessary resources are downloaded
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('wordnet_ic', quiet=True)
nltk.download('stopwords', quiet=True)

# Load the Information Content (IC) corpus
ic = wordnet_ic.ic('ic-brown.dat')

In [2]:
# Create the TextPreprocessor
preprocessor = TextPreprocessor()

# Load the training dataset
train_df = preprocessor.load_dataset('../Preprocessing/STS_train.csv')

# Display the DataFrame
train_df.head()

,0,1,gs
0,"[But, other, sources, close, to, the, sale, sa...","[But, other, sources, close, to, the, sale, sa...",4.00
1,"[Micron, has, declared, its, first, quarterly,...","[Micron, 's, numbers, also, marked, the, first...",3.75
2,"[The, fines, are, part, of, failed, Republican...","[Perry, said, he, backs, the, Senate, 's, effo...",2.80
3,"[The, American, Anglican, Council, ,, which, r...","[The, American, Anglican, Council, ,, which, r...",3.40
4,"[The, tech-loaded, Nasdaq, composite, rose, 20...","[The, technology-laced, Nasdaq, Composite, Ind...",2.40


In [3]:
# Normalize the text
normal_train_df = preprocessor.remove_punctuation(train_df)
normal_train_df = preprocessor.convert_to_lowercase(normal_train_df)
normal_train_df = preprocessor.remove_empty_strings(normal_train_df)

# Create 2 separate DataFrames, one without stopwords and the other also lemmatized
sw_train_df = preprocessor.remove_stopwords(normal_train_df)
lemmas_train_df = preprocessor.lemmatize(sw_train_df)

lemmas_train_df.head()

,0,1,gs
0,"[source, close, sale, said, vivendi, keeping, ...","[source, close, sale, said, vivendi, keeping, ...",4.00
1,"[micron, declared, first, quarterly, profit, t...","[micron, number, also, marked, first, quarterl...",3.75
2,"[fine, part, failed, republican, effort, force...","[perry, said, back, senate, effort, including,...",2.80
3,"[american, anglican, council, represents, epis...","[american, anglican, council, represents, epis...",3.40
4,"[tech-loaded, nasdaq, composite, rose, 20.96, ...","[technology-laced, nasdaq, composite, index, i...",2.40


In [4]:
# Word Sense Disambiguation

# Substitute words in a sentence based on Lesk WSD.
def substitute_words(processed_tokens, original_tokens):
    substituted_sentence = []
    for word in processed_tokens:
        # Apply Lesk algorithm using the original sentence context
        synset = lesk(original_tokens, word)
        if synset:
            # Replace with the first synonym that isn't the original word
            substitutes = [lemma.name() for lemma in synset.lemmas() if lemma.name() != word]
            substituted_sentence.append(substitutes[0] if substitutes else word)
        else:
            substituted_sentence.append(word)  # No substitution if no synset found
    return substituted_sentence

# Apply Lesk WSD to the sentences in 'df', substituting words with the most probable match in the synset.
# The original sentences are provided in 'context'.
def lesk_wsd(df, context):
    wsd_df = pd.DataFrame()

    # Apply the substitution to each sentence
    wsd_df['0'] = [substitute_words(df['0'][i], context['0'][i]) for i in range(len(df))]
    wsd_df['1'] = [substitute_words(df['1'][i], context['1'][i]) for i in range(len(df))]
    wsd_df['gs'] = df['gs']
    
    return wsd_df


wsd_train_df = lesk_wsd(lemmas_train_df, normal_train_df)
wsd_train_df.head()

,0,1,gs
0,"[reservoir, close, sales_agreement, state, viv...","[reservoir, close, sales_agreement, pronounce,...",4.00
1,"[micrometer, stated, first_base, every_quarter...","[micrometer, phone_number, besides, set, first...",3.75
2,"[fine, function, fail, republican, elbow_greas...","[perry, order, backward, United_States_Senate,...",2.80
3,"[American, Anglican, council, represent, Episc...","[American, Anglican, council, represent, Episc...",3.40
4,"[tech-loaded, National_Association_of_Securiti...","[technology-laced, National_Association_of_Sec...",2.40


In [5]:
# Group the 3 DataFrames into a Dictionary, with their "names"
train_dfs = {'normal': normal_train_df, 'sw': sw_train_df, 'lemmas': lemmas_train_df, 'wsd': wsd_train_df}

In [6]:
# Create the features DataFrame
train_features_df = pd.DataFrame()

In [7]:
# Jaccard distance
# The first 3 features are the jaccard similarity between the sentence pairs.
for name, df in train_dfs.items():
    train_features_df[f'{name}_jaccard'] = [1 - jaccard_distance(set(sentence_pair['0']), set(sentence_pair['1'])) for _, sentence_pair in df.iterrows()]

train_features_df.head()

,normal_jaccard,sw_jaccard,lemmas_jaccard,wsd_jaccard
0,0.533333,0.473684,0.473684,0.400000
1,0.388889,0.500000,0.500000,0.384615
2,0.333333,0.357143,0.357143,0.266667
3,0.607143,0.611111,0.611111,0.318182
4,0.192308,0.150000,0.150000,0.150000


In [8]:
# Containment Measure
# Apart from the Jaccard distance, we also measure the containment measure (Broder, 1997)
def containment_measure(set_a, set_b):
    # Calculate the intersection of both sets
    intersection = set_a.intersection(set_b)
    
    # Return the containment measure
    return len(intersection) / min(len(set_a), len(set_b))

for name, df in train_dfs.items():
    train_features_df[f'{name}_containment'] = [containment_measure(set(sentence_pair['0']), set(sentence_pair['1'])) for _, sentence_pair in df.iterrows()]

train_features_df.head()

,normal_jaccard,sw_jaccard,lemmas_jaccard,wsd_jaccard,normal_containment,sw_containment,lemmas_containment,wsd_containment
0,0.533333,0.473684,0.473684,0.400000,0.761905,0.750000,0.750000,0.666667
1,0.388889,0.500000,0.500000,0.384615,0.700000,0.857143,0.857143,0.714286
2,0.333333,0.357143,0.357143,0.266667,0.500000,0.555556,0.555556,0.444444
3,0.607143,0.611111,0.611111,0.318182,0.944444,0.916667,0.916667,0.583333
4,0.192308,0.150000,0.150000,0.150000,0.357143,0.272727,0.272727,0.272727


In [9]:

# Word n-grams
# That was the case for individual words (1-grams), let us now also calculate the same measures for the general n-grams
def jaccard_similarity_ngram(sentence1, sentence2, n):
    # Generate n-grams for both sentences
    ngrams1 = set(ngrams(sentence1, n))
    ngrams2 = set(ngrams(sentence2, n))

    # Handle the case when one or both sentences are too short to have any n-grams
    if not ngrams1 and not ngrams2:
        return 1  # Consider them identical if both are too short
    elif not ngrams1 or not ngrams2:
        return 0  # No overlap if one is too short
    
    # Calculate Jaccard similarity
    return 1 - jaccard_distance(ngrams1, ngrams2)


def containment_measure_ngram(sentence1, sentence2, n):
    # Generate n-grams for both sentences
    ngrams1 = set(ngrams(sentence1, n))
    ngrams2 = set(ngrams(sentence2, n))
    
    # Handle the case when one or both sentences are too short to have any n-grams
    if not ngrams1 and not ngrams2:
        return 1  # Consider them identical if both are too short
    elif not ngrams1 or not ngrams2:
        return 0  # No overlap if one is too short

    # Calculate Jaccard similarity
    return containment_measure(ngrams1, ngrams2)

for name, df in train_dfs.items():
    for n in range(2, 5):
        train_features_df[f'{name}_jaccard_{n}gram'] = [jaccard_similarity_ngram(sentence_pair['0'], sentence_pair['1'], n) for _, sentence_pair in df.iterrows()]
        train_features_df[f'{name}_containment_{n}gram'] = [containment_measure_ngram(sentence_pair['0'], sentence_pair['1'], n) for _, sentence_pair in df.iterrows()]

In [ ]:
# Pairwise Word Similarity

# Compute IDF weights for a DataFrame.
def compute_idf_weights(df):
    tokens = list(chain.from_iterable(df[['0', '1']].values.flatten()))
    token_counts = Counter(tokens)
    total_docs = len(tokens)
    return {word: math.log(total_docs / (count + 1)) for word, count in token_counts.items()}

# Compute word similarity using WordNet.
def word_similarity(word1, word2, similarity_measure):
    synsets1 = wn.synsets(word1)
    synsets2 = wn.synsets(word2)
    if not synsets1 or not synsets2:
        return 0  # Return 0 if either word has no synsets

    max_similarity = 0
    for syn1 in synsets1:
        for syn2 in synsets2:
            try:
                sim = similarity_measure(syn1, syn2)
                max_similarity = max(max_similarity, sim)
            except:
                pass
            
    if max_similarity > 20:
        return 20 # Truncate big values to avoid numerical overflow
    return max_similarity

# Calculate pairwise word similarity of a sentence with respect to other
# We use the aggregation strategy by Mihalcea et al. (2006)
def directional_similarity(src_tokens, target_tokens, similarity_measure, idf_weights):
    weighted_similarities = []
    for w in src_tokens:
        w_similarities = [word_similarity(w, target, similarity_measure) for target in target_tokens]
        weighted_w_similarity = max(w_similarities) * idf_weights.get(w, 0)
        weighted_similarities.append(weighted_w_similarity)
    numerator = sum(weighted_similarities)
    denominator = sum(idf_weights.get(w, 0) for w in src_tokens)
    return numerator / denominator if denominator > 0 else 0

# Compute sentence similarity of 2 sentences, averaging their directional sentence similarities
def sentence_similarity(t1_tokens, t2_tokens, similarity_measure, idf_weights):
    sim_t1_to_t2 = directional_similarity(t1_tokens, t2_tokens, similarity_measure, idf_weights)
    sim_t2_to_t1 = directional_similarity(t2_tokens, t1_tokens, similarity_measure, idf_weights)
    
    return 0.5 * (sim_t1_to_t2 + sim_t2_to_t1)

# Define a pipeline to compute Pairwise Word Similarity using parallelization
def compute_pairwise_word_similarities(name, df, features_df):
    idf_weights = compute_idf_weights(df)

    def compute_similarity(indexed_pair, similarity_function):
        _, sentence_pair = indexed_pair
        return sentence_similarity(sentence_pair['0'], sentence_pair['1'], similarity_function, idf_weights)

    # Resnik similarity
    with ThreadPoolExecutor() as executor:
        resnik_results = list(tqdm(
            executor.map(
                lambda indexed_pair: compute_similarity(indexed_pair, lambda syn1, syn2: syn1.res_similarity(syn2, ic)),
                df.iterrows()
            ),
            total=len(df),
            desc=f"Computing {name} Resnik similarity"
        ))
    features_df[f'{name}_resnik_similarity'] = resnik_results

    # Normalize Resnik similarity
    min_resnik_sim = min(resnik_results)
    max_resnik_sim = max(resnik_results)
    features_df[f'{name}_resnik_similarity'] = [(res - min_resnik_sim) / (max_resnik_sim - min_resnik_sim) for res in resnik_results]

    # Lin similarity
    with ThreadPoolExecutor() as executor:
        lin_results = list(tqdm(
            executor.map(
                lambda indexed_pair: compute_similarity(indexed_pair, lambda syn1, syn2: syn1.lin_similarity(syn2, ic)),
                df.iterrows()
            ),
            total=len(df),
            desc=f"Computing {name} Lin similarity"
        ))
    features_df[f'{name}_lin_similarity'] = lin_results

    # Normalize Lin similarity
    min_lin_sim = min(lin_results)
    max_lin_sim = max(lin_results)
    features_df[f'{name}_lin_similarity'] = [(lin - min_lin_sim) / (max_lin_sim - min_lin_sim) for lin in lin_results]


In [11]:
for name, df in {'lemmas': lemmas_train_df, 'wsd': wsd_train_df}.items():
    compute_pairwise_word_similarities(name, df, train_features_df)

Computing wsd Resnik similarity:   8%|▊         | 181/2234 [00:21<27:54,  1.23it/s] C:\Users\sanch\AppData\Roaming\Python\Python312\site-packages\nltk\corpus\reader\wordnet.py:1580: UserWarning: No WordNet synset found for pos=n at offset=7780173.
  warnings.warn(f"No WordNet synset found for pos={pos} at offset={offset}.")
Computing wsd Lin similarity: 100%|██████████| 2234/2234 [01:12<00:00, 30.65it/s] 


In [16]:
train_features_df['gs'] = train_df['gs'] / 5.0

train_features_df.to_csv('train/lexicalFeatures_train.csv', index=False)

# Personal Notes
Maybe delete before turn in

In [12]:
from scipy.stats import pearsonr

correlations = {}
for column in train_features_df.columns:
    corr, _ = pearsonr(train_features_df[column], train_df['gs'])
    correlations[column] = corr

# Convert the dictionary to a DataFrame for tabular representation
correlation_table = pd.DataFrame(list(correlations.items()), columns=['Variable', 'Correlation'])

print(correlation_table)

                    Variable  Correlation
0             normal_jaccard     0.476515
1                 sw_jaccard     0.600506
2             lemmas_jaccard     0.610119
3                wsd_jaccard     0.515652
4         normal_containment     0.481160
5             sw_containment     0.616511
6         lemmas_containment     0.626920
7            wsd_containment     0.541817
8       normal_jaccard_2gram     0.334975
9   normal_containment_2gram     0.354762
10      normal_jaccard_3gram     0.279903
11  normal_containment_3gram     0.286566
12      normal_jaccard_4gram     0.291721
13  normal_containment_4gram     0.311332
14          sw_jaccard_2gram     0.421352
15      sw_containment_2gram     0.443682
16          sw_jaccard_3gram     0.155031
17      sw_containment_3gram     0.219161
18          sw_jaccard_4gram    -0.304575
19      sw_containment_4gram    -0.253522
20      lemmas_jaccard_2gram     0.425503
21  lemmas_containment_2gram     0.448294
22      lemmas_jaccard_3gram     0

Consider removing 4grams, as they have negative correlation. Maybe that could be useful too, as "negative examples"?

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from scipy.stats import pearsonr

df = pd.read_csv('train/lexicalFeatures_train.csv')

# Assuming 'df' is your DataFrame, and 'gs' is the target column
X = df.drop(columns=['gs']).values
y = df['gs'].values

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the model
def create_model(input_dim):
    model = Sequential()
    model.add(Input((input_dim,)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))  # For regression, single output node
    model.compile(optimizer=Adam(), loss='mean_squared_error')
    return model

# n-fold cross-validation
n_folds = 2
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
cv_pearson_scores = []

for train_idx, val_idx in kf.split(X_scaled):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Create the model for each fold
    model = create_model(X_train.shape[1])
    
    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    # Predict on the validation set
    y_pred = model.predict(X_val)
    
    # Calculate the Pearson correlation
    corr, _ = pearsonr(y_val, y_pred.flatten())  # Flatten to ensure 1D arrays
    cv_pearson_scores.append(corr)

# Calculate average Pearson correlation across folds
avg_pearson = np.mean(cv_pearson_scores)
print(f'Average Pearson Correlation: {avg_pearson}')
print(cv_pearson_scores)

# Optionally, clear the session to free memory
K.clear_session()


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Average Pearson Correlation: 0.7574049307336346
[np.float64(0.7451099638606534), np.float64(0.7696998976066158)]
